In [1]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
boston = load_boston()
X, y = boston.data, boston.target
df_boston = pd.DataFrame(boston.data, columns=boston.feature_names)
df_boston['PRICE'] = y
print(boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

    :Attribute Information (in order):  
        - CRIM     Уровень преступности на душу населения по городу  
        - ZN       Доля жилой земли, зонированной для участков площадью более 25 000 кв. футов  
        - INDUS    Доля акров для промышленности (заводы, предприятия)  
        - CHAS     Фиктивная переменная (1, если тракт ограничивает реку; 0 в противном случае)  
        - NOX      Концентрация оксидов азота NOX (частей на 10 млн.)  
        - RM       Среднее количество комнат в одном жилом помещении  
        - AGE      Возрастная доля занятых владельцами квартир, построенных до 1940 года  
        - DIS      Взвешенные расстояния до пяти Бостонских центров занятости  
        - RAD      Индекс доступности радиальных магистралей  
        - TAX      НАЛОГ на недвижимость с полной стоимостью-ставка налога на 10 000 долларов  
        - PTRATIO  Соотношение учеников и учителей по городам  
        - B        Доля чернокожих по городам  
        - LSTAT    % населения с низким социальным статусом  
        - PRICE    Медианная стоимость домов, занятых владельцами, в $1000 долларах  

In [3]:
df_boston[:5]

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
print('Процент незаполненных данных в признаках')
for column_name in df_boston.columns:
    col_nan_stat = round(df_boston[column_name].isna().mean() * 100, 2)
    print(f'{column_name}: {col_nan_stat}%')

Процент незаполненных данных в признаках
CRIM: 0.0%
ZN: 0.0%
INDUS: 0.0%
CHAS: 0.0%
NOX: 0.0%
RM: 0.0%
AGE: 0.0%
DIS: 0.0%
RAD: 0.0%
TAX: 0.0%
PTRATIO: 0.0%
B: 0.0%
LSTAT: 0.0%
PRICE: 0.0%


Посмотрим как будет работать линейная регрессия с необработанными данными

In [5]:
scaler = StandardScaler()
lr = LinearRegression()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df_boston[df_boston.columns.drop('PRICE')],
                                                    df_boston['PRICE'], test_size=0.3,
                                                    random_state=42)
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)
lr.fit(X_train_std, y_train)
y_predict = lr.predict(X_test_std)
rmse = np.sqrt(mean_squared_error(y_test, y_predict))
r2 = r2_score(y_test, y_predict)
print('RMSE:', rmse)
print('R2:', r2)

RMSE: 4.6386899261728205
R2: 0.7112260057484932


Теперь посмотрим таблицу кореляций признаков между собой. Выбираем коэффициент корреляции Спирмена, который показывает в том числе и нелинейную зависимость признаков.

In [7]:
df_boston.corr(method='spearman')

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
CRIM,1.000000,-0.571660,0.735524,0.041537,0.821465,-0.309116,0.704140,-0.744986,0.727807,0.729045,0.465283,-0.360555,0.634760,-0.558891
ZN,-0.571660,1.000000,-0.642811,-0.041937,-0.634828,0.361074,-0.544423,0.614627,-0.278767,-0.371394,-0.448475,0.163135,-0.490074,0.438179
INDUS,0.735524,-0.642811,1.000000,0.089841,0.791189,-0.415301,0.679487,-0.757080,0.455507,0.664361,0.433710,-0.285840,0.638747,-0.578255
CHAS,0.041537,-0.041937,0.089841,1.000000,0.068426,0.058813,0.067792,-0.080248,0.024579,-0.044486,-0.136065,-0.039810,-0.050575,0.140612
NOX,0.821465,-0.634828,0.791189,0.068426,1.000000,-0.310344,0.795153,-0.880015,0.586429,0.649527,0.391309,-0.296662,0.636828,-0.562609
RM,-0.309116,0.361074,-0.415301,0.058813,-0.310344,1.000000,-0.278082,0.263168,-0.107492,-0.271898,-0.312923,0.053660,-0.640832,0.633576
AGE,0.704140,-0.544423,0.679487,0.067792,0.795153,-0.278082,1.000000,-0.801610,0.417983,0.526366,0.355384,-0.228022,0.657071,-0.547562
DIS,-0.744986,0.614627,-0.757080,-0.080248,-0.880015,0.263168,-0.801610,1.000000,-0.495806,-0.574336,-0.322041,0.249595,-0.564262,0.445857
RAD,0.727807,-0.278767,0.455507,0.024579,0.586429,-0.107492,0.417983,-0.495806,1.000000,0.704876,0.318330,-0.282533,0.394322,-0.346776
TAX,0.729045,-0.371394,0.664361,-0.044486,0.649527,-0.271898,0.526366,-0.574336,0.704876,1.000000,0.453345,-0.329843,0.534423,-0.562411


По таблице корреляции делаем следующие выводы

1. Наибольшее влияние на целевую переменную PRICE оказывают признаки LSTAT и RM, поэтому их можно назвать ключевыми признаками.
2. Признак CRIM имеет достаточную корреляцию с признаками INDUS (0.74), NOX (0.82), AGE (0.70), DIS(0.74), RAD (0.72), TAX (0.73), имеет смысл удалить эти признаки.

Пробуем удалить признаки с достаточно высокой корреляцией и посмотрим как работает линейная модель регрессии.

In [8]:
df_boston_no_corr = df_boston[df_boston.columns.drop(['INDUS', 'NOX', 'AGE', 'DIS', 'RAD', 'TAX'])]
df_boston_no_corr[:5]

,CRIM,ZN,CHAS,RM,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,0.0,6.575,15.3,396.90,4.98,24.0
1,0.02731,0.0,0.0,6.421,17.8,396.90,9.14,21.6
2,0.02729,0.0,0.0,7.185,17.8,392.83,4.03,34.7
3,0.03237,0.0,0.0,6.998,18.7,394.63,2.94,33.4
4,0.06905,0.0,0.0,7.147,18.7,396.90,5.33,36.2


In [9]:
X_train, X_test, y_train, y_test = train_test_split(df_boston_no_corr[df_boston_no_corr.columns.drop('PRICE')],
                                                    df_boston_no_corr['PRICE'], test_size=0.3,
                                                    random_state=42)
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)
lr.fit(X_train_std, y_train)
y_predict = lr.predict(X_test_std)
rmse = np.sqrt(mean_squared_error(y_test, y_predict))
r2 = r2_score(y_test, y_predict)
print('RMSE:', rmse)
print('R2:', r2)

RMSE: 5.163634482459411
R2: 0.6421686568616799


По сравнению с необработанной моделью ошибка увеличилась.

Теперь посмотрим что произойдет, если оставить только значимые с чочки зрения корреляции признаки LSTAT и RM.

In [10]:
df_boston_main = df_boston[['LSTAT', 'RM', 'PRICE']]
X_train, X_test, y_train, y_test = train_test_split(df_boston_main[df_boston_main.columns.drop('PRICE')],
                                                    df_boston_main['PRICE'], test_size=0.3,
                                                    random_state=42)
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)
lr.fit(X_train_std, y_train)
y_predict = lr.predict(X_test_std)
rmse = np.sqrt(mean_squared_error(y_test, y_predict))
r2 = r2_score(y_test, y_predict)
print('RMSE:', rmse)
print('R2:', r2)

RMSE: 5.460428346919539
R2: 0.59985184477156


Получили еще более худший результат, поэтому принимаем решение оставлять все признаки в датасете.

Теперь проанализируем на качество данных каждый признак в датасете.

In [11]:
df_boston.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


Признак CRIM имеет значительные выбросы, максимальное значение 88.976200 при значении в третьем квартиле 3.677083.

In [12]:
print('Количество образцов с CRIM > 9:', len(df_boston[df_boston['CRIM'] > 9]))

Количество образцов с CRIM > 9: 66


Так как количество строк с CRIM > 9 всего 66, а замена на медианные значения не улучшают нашу модель, удалим эти данные из датасета и проверим качество модели.

In [13]:
df_boston = df_boston[df_boston['CRIM'] <= 9]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(df_boston[df_boston.columns.drop('PRICE')],
                                                    df_boston['PRICE'], test_size=0.3,
                                                    random_state=42)
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)
lr.fit(X_train_std, y_train)
y_predict = lr.predict(X_test_std)
rmse = np.sqrt(mean_squared_error(y_test, y_predict))
r2 = r2_score(y_test, y_predict)
print('RMSE:', rmse)
print('R2:', r2)

RMSE: 3.802132689962362
R2: 0.7219662854637542


Мы получили улучшение работы модели с 4.639 до 3.802.

Признак ZN так же имеет аномальные выбросы с максимальным значением 100 при значении 3 квартиля в 12.5.

In [15]:
print('Количество образцов с ZN > 20:', len(df_boston[df_boston.ZN > 20]))

Количество образцов с ZN > 20: 101


Удаление образцов с ZN > 20 и заполнение их медианой приводит к ухудшению результата. Поэтому оставляем признак как есть.

У признака AGE так же есть выбросы в сторону минимального значения. первый квартиль имеет значение 45. А минимальное значение 2.9.

In [16]:
print('Количество образцов с AGE < 20:', len(df_boston[df_boston.AGE < 20]))

Количество образцов с AGE < 20: 34


Удаление этих данных ухудшает нашу модель по всем параметрам. Пробуем заполнить значения AGE < 30 медианным значением.

In [17]:
df_boston[df_boston.AGE < 30] = df_boston.AGE.median()

In [18]:
X_train, X_test, y_train, y_test = train_test_split(df_boston[df_boston.columns.drop('PRICE')],
                                                    df_boston['PRICE'], test_size=0.3,
                                                    random_state=42)
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)
lr.fit(X_train_std, y_train)
y_predict = lr.predict(X_test_std)
rmse = np.sqrt(mean_squared_error(y_test, y_predict))
r2 = r2_score(y_test, y_predict)
print('RMSE:', rmse)
print('R2:', r2)

RMSE: 3.711986723724966
R2: 0.9642603407813976


Значение RMSE немного улучшилось с 3.802 до 3.711, а так же значительно улучшилось качество модели по сравнению с моделью усреднения с 0.721 до 0.964.

Теперь попробуем построить модель с помощью полиномиальной регрессии. 

In [19]:
X_train, X_test, y_train, y_test = train_test_split(df_boston[df_boston.columns.drop('PRICE')],
                                                    df_boston['PRICE'], test_size=0.3,
                                                    random_state=42)

polynom = PolynomialFeatures(2)
polynom.fit(X_train)
X_train_poly = polynom.transform(X_train)
X_test_poly = polynom.transform(X_test)

scaler.fit(X_train_poly)
X_train_poly_std = scaler.transform(X_train_poly)
X_test_poly_std = scaler.transform(X_test_poly)

lr.fit(X_train_poly_std, y_train)
y_predict = lr.predict(X_test_poly_std)

rmse = np.sqrt(mean_squared_error(y_test, y_predict))
r2 = r2_score(y_test, y_predict)
print('RMSE:', rmse)
print('R2:', r2)

RMSE: 3.2839318684694314
R2: 0.9720278495691006


С помощью полинома нам удалось улучшить RMSE с 3.712 до 3.284, а R2 с 0.964 до 0.972
Найдем признаки, которые оказывают наибольшее влияние на модель полиномиальной регрессии.

In [20]:
poly_f_names = polynom.get_feature_names()
poly_f_coef = lr.coef_
poly_f_sorted_index = np.argsort(poly_f_coef)[::-1]

sorted([(np.abs(coef), name) for coef, name in zip (lr.coef_, polynom.get_feature_names())], 
       reverse=True)[:10]

[(69501.59813184377, 'x3 x4'),
 (68609.91074812133, 'x3^2'),
 (53642.05308861647, 'x4^2'),
 (28137.31751480733, 'x4 x5'),
 (22712.089876973856, 'x4 x7'),
 (12798.514508258979, 'x4 x10'),
 (7173.877170452307, 'x0 x3'),
 (5095.775604926884, 'x3 x5'),
 (4959.788391879584, 'x0 x4'),
 (4772.2809057023, 'x3 x7')]

Теперь попробуем улучшить наш датасет с помощью новых признаков из полиномиальной регрессии.

In [21]:
df_boston['CHAS*NOX'] = df_boston['CHAS'] * df_boston['NOX']
df_boston['CHAS^2'] = df_boston['CHAS'] ** 2
df_boston['NOX^2'] = df_boston['NOX'] ** 2
df_boston['NOX*RM'] = df_boston['NOX'] * df_boston['RM']
df_boston['NOX*DIS'] = df_boston['NOX'] * df_boston['DIS']
df_boston['NOX*PTRATIO'] = df_boston['NOX'] * df_boston['PTRATIO']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(df_boston[df_boston.columns.drop('PRICE')],
                                                    df_boston['PRICE'], test_size=0.3,
                                                    random_state=42)
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)
lr.fit(X_train_std, y_train)
y_predict = lr.predict(X_test_std)
rmse = np.sqrt(mean_squared_error(y_test, y_predict))
r2 = r2_score(y_test, y_predict)
print('RMSE:', rmse)
print('R2:', r2)

RMSE: 3.464966715232604
R2: 0.9688587735331491


До уровня полиномиальной регресиси добраться не удалось, но тем не менее мы RMSE с 3.712 до 3.465, а R2 с 0.964 до 0.969.

В итоге нам удалось улучшить результат с

    RMSE: 4.6386899261728205
    R2: 0.7112260057484932
до   

    RMSE: 3.464966715232604
    R2: 0.9688587735331491
    
с помощью обработки признаков и добавления к ним новых значимых признаков из полиномиальной регрессии и до 

    RMSE: 3.2839318684694314
    R2: 0.9720278495691006
    
 с помощью полиномиальной регрессии